# Preparación de datos
## Importar bibliotecas

In [1]:
import os
import polars as pl
import polars.selectors as cs
import pandas as pd
from datetime import date, time
import numpy as np
import pyarrow.parquet as pq

In [2]:
# from os import listdir
# from os.path import isfile, join

In [3]:


# Ruta al directorio que contiene los archivos Parquet
directorio = "..\\datasets\\raw\\"

# Obtener la lista de archivos en el directorio
archivos_parquet = [f for f in os.listdir(directorio) if os.path.isfile(os.path.join(directorio, f)) and f.endswith('.parquet')]

# Crear un diccionario para almacenar los DataFrames de Polars
diccionario_dataframes = {}

# Iterar sobre los archivos Parquet
for archivo in archivos_parquet:
    # Verificar si el archivo contiene "yellow", ".parquet" y "2023" en su nombre
    if 'yellow' in archivo and '.parquet' in archivo and '2022' in archivo:
        # Obtener el mes del archivo
        mes = archivo.split('_')[2].split('-')[1].split('.')[0]  # Extraer el mes del nombre del archivo
        
        # Construir la ruta completa al archivo
        ruta_archivo = os.path.join(directorio, archivo)
        
        # Leer el archivo Parquet en un DataFrame de Polars
        df = pl.read_parquet(ruta_archivo)
        
        # Agregar el DataFrame al diccionario utilizando el nombre del archivo como clave
        diccionario_dataframes[mes] = df

# Ahora el diccionario 'diccionario_dataframes' contiene un DataFrame por mes para el año 2023
# de los archivos Parquet que contienen "yellow" en su nombre.


In [4]:
# Iterar sobre el diccionario de dataframes
for nombre, df in diccionario_dataframes.items():
    print(f"Información del DataFrame '{nombre}':")
    print("Cantidad de registros :", len(df), "Cantidad de las Columnas :", len(df.columns))
    print("Schema")
    #print(df.schema)
    print("\n")

Información del DataFrame '01':
Cantidad de registros : 2463931 Cantidad de las Columnas : 19
Schema


Información del DataFrame '02':
Cantidad de registros : 2979431 Cantidad de las Columnas : 19
Schema


Información del DataFrame '03':
Cantidad de registros : 3627882 Cantidad de las Columnas : 19
Schema


Información del DataFrame '04':
Cantidad de registros : 3599920 Cantidad de las Columnas : 19
Schema


Información del DataFrame '05':
Cantidad de registros : 3588295 Cantidad de las Columnas : 19
Schema


Información del DataFrame '06':
Cantidad de registros : 3558124 Cantidad de las Columnas : 19
Schema


Información del DataFrame '07':
Cantidad de registros : 3174394 Cantidad de las Columnas : 19
Schema


Información del DataFrame '08':
Cantidad de registros : 3152677 Cantidad de las Columnas : 19
Schema


Información del DataFrame '09':
Cantidad de registros : 3183767 Cantidad de las Columnas : 19
Schema


Información del DataFrame '10':
Cantidad de registros : 3675411 Cantidad 

In [5]:
# Supongamos que diccionario_dataframes es tu diccionario que contiene los DataFrames
# Reemplaza diccionario_dataframes con el nombre de tu diccionario
"""
# Tomar el primer DataFrame del diccionario
primer_df = next(iter(diccionario_dataframes.values()))

# Definir un nuevo esquema con los tipos de datos y nombres de columna deseados
nuevo_esquema = {
    "VendorID": pl.Int32,
    "tpep_pickup_datetime": pl.Datetime(time_unit='ns', time_zone=None),
    "tpep_dropoff_datetime": pl.Datetime(time_unit='ns', time_zone=None),
    "passenger_count": pl.Int64,
    "trip_distance": pl.Float64,
    "RatecodeID": pl.Int64,
    "store_and_fwd_flag": pl.String,
    "PULocationID": pl.Int32,
    "DOLocationID": pl.Int32,
    "payment_type": pl.Int64,
    "fare_amount": pl.Float64,
    "extra": pl.Float64,
    "mta_tax": pl.Float64,
    "tip_amount": pl.Float64,
    "tolls_amount": pl.Float64,
    "improvement_surcharge": pl.Float64,
    "total_amount": pl.Float64,
    "congestion_surcharge": pl.Float64,
    "airport_fee": pl.Float64,  # He cambiado el nombre de la columna a "airport_fee"
}

# Renombrar las columnas y cambiar los tipos de datos del DataFrame
df = primer_df.select([
    pl.col(col).cast(dtype).alias(col) for col, dtype in nuevo_esquema.items()
])

# Cambiar el nombre de la columna "airport_fee" a "Airport_fee"
df = df.with_columns(primer_df['airport_fee'].alias('Airport_fee'))
df = df.drop('airport_fee')
# Reemplazar el DataFrame original en el diccionario con el DataFrame modificado
nombre_clave = next(iter(diccionario_dataframes))  # Obtener la clave del primer DataFrame
diccionario_dataframes[nombre_clave] = df

# Verificar el resultado
print(diccionario_dataframes[nombre_clave].schema)

"""

'\n# Tomar el primer DataFrame del diccionario\nprimer_df = next(iter(diccionario_dataframes.values()))\n\n# Definir un nuevo esquema con los tipos de datos y nombres de columna deseados\nnuevo_esquema = {\n    "VendorID": pl.Int32,\n    "tpep_pickup_datetime": pl.Datetime(time_unit=\'ns\', time_zone=None),\n    "tpep_dropoff_datetime": pl.Datetime(time_unit=\'ns\', time_zone=None),\n    "passenger_count": pl.Int64,\n    "trip_distance": pl.Float64,\n    "RatecodeID": pl.Int64,\n    "store_and_fwd_flag": pl.String,\n    "PULocationID": pl.Int32,\n    "DOLocationID": pl.Int32,\n    "payment_type": pl.Int64,\n    "fare_amount": pl.Float64,\n    "extra": pl.Float64,\n    "mta_tax": pl.Float64,\n    "tip_amount": pl.Float64,\n    "tolls_amount": pl.Float64,\n    "improvement_surcharge": pl.Float64,\n    "total_amount": pl.Float64,\n    "congestion_surcharge": pl.Float64,\n    "airport_fee": pl.Float64,  # He cambiado el nombre de la columna a "airport_fee"\n}\n\n# Renombrar las columnas y 

In [6]:
# Iterar sobre el diccionario de dataframes
# for nombre, df in diccionario_dataframes.items():
#     print(f"Información del DataFrame '{nombre}':")
#     print("Cantidad de registros :", len(df), "Cantidad de las Columnas :", len(df.columns))
#     print("Schema")
#     print(df.schema)
#     print("\n")

In [7]:
# Ahora que todos los DataFrames tienen las mismas columnas, puedes proceder a concatenarlos.
dataframes = list(diccionario_dataframes.values())

# Concatenar verticalmente todos los DataFrames
df_concatenado = pl.concat(dataframes)

In [8]:
df_concatenado.shape

(39656098, 19)

In [9]:
df_concatenado.schema

OrderedDict([('VendorID', Int64),
             ('tpep_pickup_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('tpep_dropoff_datetime',
              Datetime(time_unit='ns', time_zone=None)),
             ('passenger_count', Float64),
             ('trip_distance', Float64),
             ('RatecodeID', Float64),
             ('store_and_fwd_flag', String),
             ('PULocationID', Int64),
             ('DOLocationID', Int64),
             ('payment_type', Int64),
             ('fare_amount', Float64),
             ('extra', Float64),
             ('mta_tax', Float64),
             ('tip_amount', Float64),
             ('tolls_amount', Float64),
             ('improvement_surcharge', Float64),
             ('total_amount', Float64),
             ('congestion_surcharge', Float64),
             ('airport_fee', Float64)])

In [10]:
df_concatenado.head()

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
i64,datetime[ns],datetime[ns],f64,f64,f64,str,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64
1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.8,1.0,"""N""",142,236,1,14.5,3.0,0.5,3.65,0.0,0.3,21.95,2.5,0.0
1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.1,1.0,"""N""",236,42,1,8.0,0.5,0.5,4.0,0.0,0.3,13.3,0.0,0.0
2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,"""N""",166,166,1,7.5,0.5,0.5,1.76,0.0,0.3,10.56,0.0,0.0
2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,"""N""",114,68,2,8.0,0.5,0.5,0.0,0.0,0.3,11.8,2.5,0.0
2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.3,1.0,"""N""",68,163,1,23.5,0.5,0.5,3.0,0.0,0.3,30.3,2.5,0.0


In [11]:
# Obtener el recuento de valores nulos como un objeto lazy
lazy_result = df_concatenado.lazy().null_count()

# Ejecutar la operación lazy y obtener el resultado
result = lazy_result.collect()

# Mostrar el resultado
result

VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,1368303,0,1368303,1368303,0,0,0,0,0,0,0,0,0,0,1368303,1368303


In [12]:
# Obtener solo las columnas numéricas 
numeric_columns = [col_name for col_name in df_concatenado.columns if df_concatenado[col_name].dtype.is_numeric()]

# Convertir la selección de columnas en una operación lazy
lazy_select = df_concatenado.select(numeric_columns)

# Calcular estadísticas descriptivas solo para las columnas numéricas de manera lazy
lazy_stats = lazy_select.describe()

lazy_stats

statistic,VendorID,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",3.9656098e7,3.8287795e7,3.9656098e7,3.8287795e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.9656098e7,3.8287795e7,3.8287795e7
"""null_count""",0.0,1.368303e6,0.0,1.368303e6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.368303e6,1.368303e6
"""mean""",1.721802,1.401149,5.959399,1.424172,164.865988,162.575201,1.18955,10.362822,1.007532,0.488806,7.234908,0.536283,0.316124,21.671268,2.281437,0.097419
"""std""",0.480527,0.962894,599.190714,5.794343,65.310815,70.231459,0.519041,22328.299433,1.262564,0.093804,22328.083995,2.043351,0.129332,96.373602,0.751943,0.338539
"""min""",1.0,0.0,0.0,1.0,1.0,1.0,0.0,-1.3339e8,-22.18,-0.55,-410.0,-99.99,-1.0,-2567.8,-2.5,-1.25
"""25%""",1.0,1.0,1.1,1.0,132.0,113.0,1.0,7.0,0.0,0.5,0.93,0.0,0.3,12.3,2.5,0.0
"""50%""",2.0,1.0,1.9,1.0,162.0,162.0,1.0,10.5,0.5,0.5,2.16,0.0,0.3,15.96,2.5,0.0
"""75%""",2.0,1.0,3.56,1.0,234.0,234.0,1.0,16.5,2.5,0.5,3.36,0.0,0.3,23.16,2.5,0.0
"""max""",6.0,9.0,389678.46,99.0,265.0,265.0,5.0,401092.32,33.5,25.48,1.3339e8,911.87,1.0,401095.62,2.75,1.25


In [13]:
# Eliminar las columnas originales de fecha y hora
df_nyc = df_concatenado.drop(['store_and_fwd_flag', 'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge', 'congestion_surcharge', 'airport_fee'])

In [14]:
df_nyc = df_nyc.drop_nulls()

In [15]:
df_nyc.shape

(38287795, 10)

## General Preprocessing `Yellow Taxi Trip Records`
### Preprocesamiento Variables de Tipo Fecha `tpep_pickup_datetime` y  `tpep_dropoff_datetime`

In [16]:
# Crear un contexto SQL y registrar el DataFrame como una tabla
sql = pl.SQLContext(frames={"df_nyc": df_nyc})

# Realizar las consultas de forma "Lazy"
result_lazy = sql.execute("SELECT * FROM df_nyc LIMIT 15")

# Convertir los resultados "Lazy" a DataFrames de Pandas solo cuando se necesiten
result_df = result_lazy.collect().to_pandas()

# Mostrar los DataFrames resultantes
result_df.head(15)


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,PULocationID,DOLocationID,payment_type,total_amount
0,1,2022-01-01 00:35:40,2022-01-01 00:53:29,2.0,3.80,1.0,142,236,1,21.95
1,1,2022-01-01 00:33:43,2022-01-01 00:42:07,1.0,2.10,1.0,236,42,1,13.30
2,2,2022-01-01 00:53:21,2022-01-01 01:02:19,1.0,0.97,1.0,166,166,1,10.56
3,2,2022-01-01 00:25:21,2022-01-01 00:35:23,1.0,1.09,1.0,114,68,2,11.80
4,2,2022-01-01 00:36:48,2022-01-01 01:14:20,1.0,4.30,1.0,68,163,1,30.30
5,1,2022-01-01 00:40:15,2022-01-01 01:09:48,1.0,10.30,1.0,138,161,1,56.35
6,2,2022-01-01 00:20:50,2022-01-01 00:34:58,1.0,5.07,1.0,233,87,1,26.00
7,2,2022-01-01 00:13:04,2022-01-01 00:22:45,1.0,2.02,1.0,238,152,2,12.80
8,2,2022-01-01 00:30:02,2022-01-01 00:44:49,1.0,2.71,1.0,166,236,1,18.05
9,2,2022-01-01 00:48:52,2022-01-01 00:53:28,1.0,0.78,1.0,236,141,2,8.80


In [17]:
from datetime import datetime

In [18]:
# Convertir las columnas de fechas a tipo DateTime en Polars
df_nyc = df_nyc.with_columns(
    pl.col('tpep_pickup_datetime').dt.date().alias('tpep_pickup_date'),
    pl.col('tpep_dropoff_datetime').dt.date().alias('tpep_dropoff_date')
)

# Calcular la duración del viaje en segundos
df_nyc = df_nyc.with_columns(
    (pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds().alias('duracion_viaje')
)
# Eliminar las columnas originales de fecha y hora
df_nyc = df_nyc.drop(['tpep_pickup_datetime', 'tpep_dropoff_datetime'])


In [19]:
df_nyc.shape

(38287795, 11)

In [20]:
# Convertir el DataFrame de Polars a un objeto Arrow
#arrow_table = df_nyc.to_arrow()


In [21]:

# Guardar el objeto Arrow en un archivo Arrow
#pq.write_table(arrow_table, 'temperature.arrow')
#pq.write_table(arrow_table, '../datasets/processed/df_yellow.arrow')
#pq.write_table(arrow_table, '../datasets/processed/yellow_analys.arrow', compression='zstd')

In [22]:
# Eliminación de registros que están fuera de la normativa legal
df_nyc = df_nyc.filter(
    (pl.col('duracion_viaje') > 208) & (pl.col('duracion_viaje') <= 2393)
)

: 

### Preprocesamiento Variables Discretas `passenger_count`.


- **Count**: El recuento total de observaciones para la variable `passenger_count` es aproximadamente 38,287,795.
- **Null Count**: Hay alrededor de 1,368,303 valores nulos en la variable `passenger_count`.
- **Mean (Media)**: La media de `passenger_count` es aproximadamente 1.401, lo que sugiere que, en promedio, hay alrededor de 1.4 pasajeros por viaje de taxi.
- **Standard Deviation (Desviación estándar)**: La desviación estándar es aproximadamente 0.962, lo que indica que la dispersión de los valores alrededor de la media es relativamente baja, lo que sugiere que la mayoría de los viajes tienen un número similar de pasajeros.
- **Min (Mínimo)**: El valor mínimo observado para `passenger_count` es 0, lo que podría ser una anomalía o un error en los datos, ya que no se esperaría que haya viajes de taxi sin pasajeros.
- **25% (Percentil 25)**: El 25% de los viajes tienen 1 pasajero o menos.
- **50% (Percentil 50 o Mediana)**: La mediana de `passenger_count` es 1, lo que significa que el 50% de los viajes tienen 1 pasajero o menos.
- **75% (Percentil 75)**: El 75% de los viajes tienen 1 o 2 pasajeros.
- **Max (Máximo)**: El valor máximo observado es 9, lo que sugiere que hay algunos viajes de taxi que transportan hasta 9 pasajeros.

En resumen, la mayoría de los viajes de taxi tienen uno o pocos pasajeros, con un promedio de alrededor de 1.4 pasajeros por viaje. Sin embargo, hay algunas observaciones inusuales, como viajes sin pasajeros (0) y algunos viajes con un número inusualmente alto de pasajeros (hasta 9).

In [ ]:
# Eliminación de registros que están fuera de la normativa legal
df_nyc = df_nyc.filter(
    (pl.col('passenger_count') > 0) & (pl.col('passenger_count') <= 4)
)

In [ ]:
# Corroboramos los cambios aplicados
unique_passenger_counts = df_nyc['passenger_count'].unique()
unique_passenger_counts 

passenger_count
f64
1.0
2.0
3.0
4.0


In [ ]:
df_nyc.shape

(36378698, 12)

### Preprocesamiento Variables Continuas `trip_distance` y `total_amount`.

- **Count**: El recuento total de observaciones para la variable `trip_distance` es aproximadamente 39,656,098.
- **Null Count**: No hay valores nulos en la variable `trip_distance`, lo que significa que todos los registros tienen una distancia de viaje registrada.
- **Mean (Media)**: La media de `trip_distance` es aproximadamente 5.959 millas. Esto indica que, en promedio, los viajes de taxi tienen una distancia de alrededor de 5.959 millas.
- **Standard Deviation (Desviación estándar)**: La desviación estándar es aproximadamente 599.191 millas, lo que sugiere que hay una gran variabilidad en las distancias de los viajes de taxi, con algunos viajes siendo mucho más largos o más cortos que la media.
- **Min (Mínimo)**: El valor mínimo observado para `trip_distance` es 0 millas, lo que podría ser una anomalía o un error en los datos, ya que no se esperaría que haya viajes de taxi con distancia cero.
- **25% (Percentil 25)**: El 25% de los viajes tienen una distancia de 1.1 millas o menos.
- **50% (Percentil 50 o Mediana)**: La mediana de `trip_distance` es 1.9 millas, lo que significa que el 50% de los viajes tienen una distancia de 1.9 millas o menos.
- **75% (Percentil 75)**: El 75% de los viajes tienen una distancia de 3.56 millas o menos.
- **Max (Máximo)**: El valor máximo observado es 389,678.46 millas, lo que sugiere que hay algunos viajes de taxi con distancias extremadamente largas.

En resumen, la mayoría de los viajes de taxi tienen distancias relativamente cortas, con una media de aproximadamente 5.959 millas y una mediana de 1.9 millas. Sin embargo, hay una variabilidad considerable en las distancias de los viajes, con algunos viajes siendo mucho más largos que la media. 

In [ ]:
"""
# Definir la condición para marcar los registros que cumplen con los criterios de filtrado
condicion = (pl.col('trip_distance') < 0.5) | (pl.col('trip_distance') > 13)

# Marcar los registros que cumplen con la condición y crear una nueva columna
df_nyc = df_nyc.with_columns([
    pl.when(condicion).then(1).otherwise(0).alias("filtro_trip_distance")
])

# Contar los registros con duración de viaje inferior a 0.5 o superiores a 13
cantidad_registros_filtrados = df_nyc.filter(pl.col("filtro_trip_distance") == 1).height
print("Cantidad de registros con trip_distance inferior a 0.5 o superior a 13:", cantidad_registros_filtrados)
#Cantidad de registros con trip_distance inferior a 0.5 o superior a 13: 3731894
"""

Cantidad de registros con trip_distance inferior a 0.5 o superior a 13: 3731894


In [ ]:
# Eliminación de registros que están fuera de la normativa legal
df_nyc = df_nyc.filter(
    (pl.col('trip_distance') > 0.5) & (pl.col('trip_distance') <= 13)
)

In [ ]:
df_nyc.shape

In [ ]:
# Convertir la columna 'trip_distance' a una expresión perezosa
trip_distance_series = df_nyc.lazy().select(pl.col('trip_distance'))

# Calcular las estadísticas de forma "Lazy"
stats_result = trip_distance_series.describe()

# Imprimir los resultados
print("Minimo:", stats_result['trip_distance'].min())
print("Maximo:", stats_result['trip_distance'].max())
print("Media:", stats_result['trip_distance'].mean())
print("Mediana:", stats_result['trip_distance'].quantile(0.5))
print("Desviación estándar:", stats_result['trip_distance'].std())


Para interpretar y analizar la columna `total_amount`, podemos observar las estadísticas proporcionadas:

- **Count**: La cantidad total de registros para esta columna es de 39,656,098.
- **Null Count**: Hay 1,368,303 registros con valores nulos para esta columna.
- **Mean**: El valor medio o promedio de la columna `total_amount` es de aproximadamente 21.67.
- **Std**: La desviación estándar, que mide la dispersión de los valores respecto a la media, es de aproximadamente 96.37.
- **Min**: El valor mínimo registrado en esta columna es -2567.8, lo que sugiere la presencia de valores negativos.
- **25%**: El 25% de los registros tienen un valor de `total_amount` de 12.3 o menos.
- **50% (Median)**: El 50% de los registros tienen un valor de `total_amount` de 15.96 o menos.
- **75%**: El 75% de los registros tienen un valor de `total_amount` de 23.16 o menos.
- **Max**: El valor máximo registrado en esta columna es 401095.62.

#### Análisis e Interpretación:

1. **Distribución de Valores**: La columna `total_amount` parece tener una amplia gama de valores, con un rango desde valores negativos hasta valores muy altos (más de 401,000). Esto sugiere una distribución muy dispersa de los montos totales de los viajes.

2. **Valores Atípicos**: La presencia de valores negativos y valores muy altos puede indicar la presencia de datos atípicos o errores en la captura de datos.

3. **Promedio y Desviación Estándar**: El promedio (`mean`) y la desviación estándar (`std`) son útiles para comprender la dispersión de los datos. Dado el valor alto de la desviación estándar en comparación con la media, podemos inferir que los datos están muy dispersos alrededor de la media, lo que puede ser indicativo de la presencia de valores atípicos o una distribución sesgada.

4. **Valores Percentiles**: Los percentiles proporcionan información sobre la distribución de los valores en la columna. Por ejemplo, el percentil 25 (25%) sugiere que el 25% de los registros tienen un valor de `total_amount` de 12.3 o menos, mientras que el percentil 75 (75%) indica que el 75% de los registros tienen un valor de 23.16 o menos. Esto ayuda a comprender la distribución de los datos en relación con diferentes rangos de valores.

5. **Valor Mínimo y Máximo**: Los valores mínimo y máximo proporcionan información sobre los extremos de la distribución de los datos. La presencia de valores negativos y el valor máximo inusualmente alto pueden requerir una revisión adicional para comprender su naturaleza y posible impacto en el análisis.

In [ ]:
# Eliminación de registros que están fuera de la normativa legal
df_nyc = df_nyc.filter(
    (pl.col('total_amount') > 10.3) & (pl.col('total_amount') <= 44.35)
)

In [ ]:
df_nyc.shape

In [ ]:
# Convertir la columna 'trip_distance' a una expresión perezosa
trip_distance_series = df_nyc.lazy().select(pl.col('total_amount'))

# Calcular las estadísticas de forma "Lazy"
stats_result = trip_distance_series.describe()

# Imprimir los resultados
print("Minimo:", stats_result['total_amount'].min())
print("Maximo:", stats_result['total_amount'].max())
print("Media:", stats_result['total_amount'].mean())
print("Mediana:", stats_result['total_amount'].quantile(0.5))
print("Desviación estándar:", stats_result['total_amount'].std())

### Preprocesamiento Variables Nominales `VendorID`, `PULocationID`, `DOLocationID` y `payment_type`.

### Preprocesamiento Variables Nominales `RatecodeID`.

In [ ]:
# Convertir el DataFrame de Polars a un objeto Arrow
arrow_table = df_nyc.to_arrow()

# Guardar el objeto Arrow en un archivo Arrow
#pq.write_table(arrow_table, 'temperature.arrow')
#pq.write_table(arrow_table, '../datasets/processed/df_yellow.arrow')
pq.write_table(arrow_table, '../datasets/processed/df_yellow.arrow', compression='zstd')

## PreProcesamiento `High Volume FHV Trip Records`

In [4]:
# Define the Imports 
try:
    import os
    import json
    import math
    from datetime import date, time
    import dask
    import pyarrow as pa
    import pyarrow.parquet as pq
    from dask.distributed import Client, LocalCluster
    import dask.dataframe as dd
    import numpy as np
    import dask.multiprocessing
except Exception as e:
    print("Some Modules are Missing : {} ".format(e))

In [5]:
size = os.path.getsize("..\\datasets\\raw\\fhvhv_tripdata_2022-01.parquet") / math.pow(1024,3)
print("Size in GB : {} ".format(size))

Size in GB : 0.34889130666852 


In [7]:
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://192.168.100.11:8787/status,
Dashboard: http://192.168.100.11:8787/status,Workers: 2
Total threads: 2,Total memory: 9.31 GiB
Status: running,Using processes: False
Comm: inproc://192.168.100.11/8444/1,Workers: 2
Dashboard: http://192.168.100.11:8787/status,Total threads: 2
Started: Just now,Total memory: 9.31 GiB
Comm: inproc://192.168.100.11/8444/4,Total threads: 1
Dashboard: http://192.168.100.11:57368/status,Memory: 4.66 GiB
Nanny: None,


In [4]:
#client.cluster.workers
cluster.dashboard_link

'http://127.0.0.1:8787/status'

In [12]:
df_dask

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,tolls,bcf,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
npartitions=1,,,,,,,,,,,,,,,,,,,,,,,,
,string,string,string,datetime64[us],datetime64[us],datetime64[us],datetime64[us],int64,int64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,string,string,string,string,string
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [12]:
# Obtener el número de columnas
columnas = len(df_dask.columns)

# Obtener el número de filas
num_filas = df_dask.shape[0].compute()

# Imprimir el número de filas y columnas
print("Número de filas:", num_filas)
print("Número de columnas:", columnas)

Número de filas: 14751591.0
Número de columnas: 24


## General Preprocessing `High Volume FHV Trip Records`


In [13]:
# Obtener el esquema de tipos de datos (dtype) de cada columna
schema_dask = df_dask.dtypes
print(schema_dask)

hvfhs_license_num       string[pyarrow]
dispatching_base_num    string[pyarrow]
originating_base_num    string[pyarrow]
request_datetime         datetime64[us]
on_scene_datetime        datetime64[us]
pickup_datetime          datetime64[us]
dropoff_datetime         datetime64[us]
PULocationID                      int64
DOLocationID                      int64
trip_miles                      float64
trip_time                         int64
base_passenger_fare             float64
tolls                           float64
bcf                             float64
sales_tax                       float64
congestion_surcharge            float64
airport_fee                     float64
tips                            float64
driver_pay                      float64
shared_request_flag     string[pyarrow]
shared_match_flag       string[pyarrow]
access_a_ride_flag      string[pyarrow]
wav_request_flag        string[pyarrow]
wav_match_flag          string[pyarrow]
dtype: object


In [6]:
# Obtener los primeros 15 registros del DataFrame de Dask
df_dask.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,sales_tax,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag
0,HV0003,B03404,B03404,2022-01-01 00:05:31,2022-01-01 00:05:40,2022-01-01 00:07:24,2022-01-01 00:18:28,170,161,1.18,...,2.21,2.75,0.0,0.0,23.03,N,N,,N,N
1,HV0003,B03404,B03404,2022-01-01 00:19:27,2022-01-01 00:22:08,2022-01-01 00:22:32,2022-01-01 00:30:12,237,161,0.82,...,1.06,2.75,0.0,0.0,12.32,N,N,,N,N
2,HV0003,B03404,B03404,2022-01-01 00:43:53,2022-01-01 00:57:37,2022-01-01 00:57:37,2022-01-01 01:07:32,237,161,1.18,...,2.65,2.75,0.0,0.0,23.30,N,N,,N,N
3,HV0003,B03404,B03404,2022-01-01 00:15:36,2022-01-01 00:17:08,2022-01-01 00:18:02,2022-01-01 00:23:05,262,229,1.65,...,0.70,2.75,0.0,0.0,6.30,N,N,,N,N
4,HV0003,B03404,B03404,2022-01-01 00:25:45,2022-01-01 00:26:01,2022-01-01 00:28:01,2022-01-01 00:35:42,229,141,1.65,...,0.84,2.75,0.0,0.0,7.44,N,N,,N,N


In [14]:
# Eliminar las columnas originales de fecha y hora
columnas_a_eliminar = ['originating_base_num', 'access_a_ride_flag', 'wav_request_flag', 'wav_match_flag', 'shared_request_flag', 'shared_match_flag','access_a_ride_flag']
df_dask = df_dask.drop(columnas_a_eliminar, axis=1)
# Llamar a compute para ejecutar la operación y modificar el DataFrame
#df_dask = df_dask.compute()

In [15]:
# Obtener el número de columnas
columnas = len(df_dask.columns)

# Obtener el número de filas
num_filas = df_dask.shape[0].compute()

# Imprimir el número de filas y columnas
print("Número de filas:", num_filas)
print("Número de columnas:", columnas)

Número de filas: 14751591.0
Número de columnas: 18


### Preprocesamiento Variables de Tipos Fechas `pickup_datetime`, `dropoff_datetime`,  `request_datetime` y  `on_scene_datetime`

In [16]:
# Convertir las columnas de fechas a tipo DateTime en Dask
df_dask['pickup_datetime'] = dd.to_datetime(df_dask['pickup_datetime'])
df_dask['dropoff_datetime'] = dd.to_datetime(df_dask['dropoff_datetime'])

In [17]:
# Calcular la duración del viaje en segundos
df_dask['viaje_segundos'] = (df_dask['dropoff_datetime'] - df_dask['pickup_datetime']).dt.total_seconds()

In [18]:
# Eliminar las columnas originales de fecha y hora
df_dask = df_dask.drop(['pickup_datetime', 'dropoff_datetime'], axis=1)

In [19]:
# Obtener el número de columnas
columnas = len(df_dask.columns)

# Obtener el número de filas
num_filas = df_dask.shape[0].compute()

# Imprimir el número de filas y columnas
print("Número de filas:", num_filas)
print("Número de columnas:", columnas)

Número de filas: 14751591.0
Número de columnas: 17


In [20]:
# Convertir las columnas de fechas a tipo DateTime en Dask
df_dask['request_datetime'] = dd.to_datetime(df_dask['request_datetime'])
df_dask['on_scene_datetime'] = dd.to_datetime(df_dask['on_scene_datetime'])

In [21]:
# Calcular la duración de espera en segundos
df_dask['espera_segundos'] = (df_dask['on_scene_datetime'] - df_dask['request_datetime']).dt.total_seconds()

In [15]:
# Eliminar las columnas originales de fecha y hora
#df_dask = df_dask.drop(['request_datetime', 'on_scene_datetime'], axis=1)

In [22]:
# Obtener el número de columnas
columnas = len(df_dask.columns)

# Obtener el número de filas
num_filas = df_dask.shape[0].compute()

# Imprimir el número de filas y columnas
print("Número de filas:", num_filas)
print("Número de columnas:", columnas)

Número de filas: 14751591.0
Número de columnas: 18


### Preprocesamiento Variables de Tipos Númericas

In [23]:
# Reemplazar los valores nulos por 0.00 en las columnas relevantes
relevant_columns = ['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge',
                    'airport_fee', 'tips', 'driver_pay']

# Reemplazar los valores nulos por 0.00 en todas las columnas relevantes
for col in relevant_columns:
    df_dask[col] = df_dask[col].fillna(0.00)

In [24]:
relevant_columns = ['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay']
df_dask['total_amount'] = df_dask[relevant_columns].sum(axis=1)

In [25]:
# Eliminar las columnas originales de fecha y hora
columnas_a_eliminar = ['tolls', 'bcf', 'sales_tax', 'congestion_surcharge','airport_fee', 'tips', 'driver_pay']
df_dask = df_dask.drop(columnas_a_eliminar, axis=1)

In [26]:
# Obtener el número de columnas
columnas = len(df_dask.columns)

# Obtener el número de filas
num_filas = df_dask.shape[0].compute()

# Imprimir el número de filas y columnas
print("Número de filas:", num_filas)
print("Número de columnas:", columnas)

Número de filas: 14751591.0
Número de columnas: 12


In [22]:
df_dask.head()

,hvfhs_license_num,dispatching_base_num,PULocationID,DOLocationID,trip_miles,trip_time,base_passenger_fare,viaje_segundos,espera_segundos,total_amount
0,HV0003,B03404,170,161,1.18,664,24.90,664.0,9.0,53.64
1,HV0003,B03404,237,161,0.82,460,11.97,460.0,161.0,28.46
2,HV0003,B03404,237,161,1.18,595,29.82,595.0,824.0,59.41
3,HV0003,B03404,262,229,1.65,303,7.91,303.0,92.0,17.90
4,HV0003,B03404,229,141,1.65,461,9.44,461.0,16.0,20.75


In [27]:
#df_pyarrow = df_dask.map_partitions(pa.Table.from_pandas).compute()
#df_final_computado = df_dask.compute()
#df_final_computado

In [ ]:
#print("Número de particiones:", df_final_computado.npartitions)


In [1]:
from dask.distributed import Client, LocalCluster
import dask.dataframe as dd
import multiprocessing

# Número de núcleos de CPU disponibles
num_cores = multiprocessing.cpu_count()
num_cores

4

In [5]:
# 4. Dividir el DataFrame en particiones y escribir en archivos Parquet
particiones = df_dask.to_delayed()
ruta_salida = '../datasets/processed/ffvh_analys_part'
for i, particion in enumerate(particiones):
    particion.to_parquet(f'{ruta_salida}{i}.parquet', engine='pyarrow')

In [7]:
# 6. Leer los archivos Parquet procesados (si es necesario)
archivos_procesados = f'{ruta_salida}*.parquet'
df_procesado = dd.read_parquet(archivos_procesados, engine='pyarrow')

In [3]:
import dask.dataframe as dd
from dask.distributed import Client, LocalCluster

# Configurar el clúster de Dask
cluster = LocalCluster(n_workers=2, threads_per_worker=1, memory_limit='12GB')
client = Client(cluster)

# Definir una función de procesamiento avanzado por partición
def procesamiento_avanzado_particion(df):
    # Filtrar y eliminar columnas no deseadas
    columnas_a_eliminar = ['originating_base_num', 'access_a_ride_flag', 'wav_request_flag', 
                           'wav_match_flag', 'shared_request_flag', 'shared_match_flag',
                           'access_a_ride_flag']
    df = df.drop(columnas_a_eliminar, axis=1)

    # Convertir columnas de fechas a tipo DateTime en Dask
    for col in ['pickup_datetime', 'dropoff_datetime', 'request_datetime', 'on_scene_datetime']:
        df[col] = dd.to_datetime(df[col])

    # Agregar columnas para fechas, horas, minutos y segundos
    for col in ['pickup_datetime', 'dropoff_datetime', 'request_datetime', 'on_scene_datetime']:
        df[col + '_fecha'] = df[col].dt.date
        df[col + '_hora_minuto'] = df[col].dt.strftime('%H:%M')  # Formatear como HH:MM
    # Calcular la duración del viaje y de espera en segundos
    df['duracion_viaje'] = (df['dropoff_datetime'] - df['pickup_datetime']).dt.total_seconds()
    df['duracion_atencion'] = (df['on_scene_datetime'] - df['request_datetime']).dt.total_seconds()

    # Eliminar filas con duraciones negativas o nulas
    df = df[df['duracion_viaje'] > 0]
    df = df[df['duracion_atencion'] >= 0]
    # Eliminar columnas originales
    df = df.drop(columns=['pickup_datetime', 'dropoff_datetime', 'request_datetime', 'on_scene_datetime'])
    # Reemplazar valores nulos por 0.00 en columnas relevantes
    relevant_columns = ['base_passenger_fare', 'tolls', 'bcf', 'sales_tax', 'congestion_surcharge',
                        'airport_fee', 'tips', 'driver_pay']
    df = df.fillna({col: 0.00 for col in relevant_columns})

    # Calcular la columna 'total_amount' sumando las columnas relevantes
    df['total_amount'] = df[relevant_columns].sum(axis=1)

    # Eliminar columnas no deseadas
    columnas_a_eliminar = ['tolls', 'bcf', 'sales_tax', 'congestion_surcharge', 
                           'airport_fee', 'tips', 'driver_pay']
    df = df.drop(columnas_a_eliminar, axis=1)

    return df

# Cargar los datos en un DataFrame distribuido de Dask y particionarlo
directorio = "../datasets/raw/fhvhv_tripdata_2022-01.parquet"
df_dask = dd.read_parquet(directorio, engine='pyarrow')
df_dask_particionado = df_dask.repartition(npartitions=4)

# Aplicar el procesamiento avanzado a cada partición y escribir en parquet
for i, particion in enumerate(df_dask_particionado.to_delayed()):
    df_particion_procesado = procesamiento_avanzado_particion(particion.compute())
    ruta_salida = f'../datasets/processed/ffvh_analys/ffvh_analys_part_{i}.parquet'
    df_particion_procesado.to_parquet(ruta_salida, engine='pyarrow')

C:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\distributed\node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 50555 instead
  warnings.warn(
C:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\distributed\client.py:3157: UserWarning: Sending large graph of size 28.14 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
C:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\distributed\client.py:3157: UserWarning: Sending large graph of size 28.14 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(
C:\Users\ozi\AppData\Local\Programs\Python\Python310\lib\site-packages\distributed\client.py:3157: UserWarning: Sending large graph of size 28.14 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  wa

In [4]:
# 5. Cerrar el clúster de Dask
client.close()
cluster.close()